In [13]:
# train a Deep Learning model on MNIST Data to recognize digits in images using pytorch
# and compute the carbon footprint of the training process

In [14]:
# import libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from codecarbon import OfflineEmissionsTracker

In [15]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [16]:
# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [17]:
# set the batch size
batch_size = 64

In [18]:
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [19]:
# creating model
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [20]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
# optimizer and loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [23]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
# import the tracker using explicit mode
tracker = OfflineEmissionsTracker(country_iso_code="ITA")
tracker.start()

epochs = 500
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

emissions: float = tracker.stop()
print(emissions)

[codecarbon INFO @ 11:21:09] offline tracker init
[codecarbon INFO @ 11:21:09] [setup] RAM Tracking...
[codecarbon INFO @ 11:21:09] [setup] GPU Tracking...
[codecarbon INFO @ 11:21:09] No GPU found.
[codecarbon INFO @ 11:21:09] [setup] CPU Tracking...
[codecarbon WARNING @ 11:21:09] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 11:21:09] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 11:21:09] >>> Tracker's metadata:
[codecarbon INFO @ 11:21:09]   Platform system: macOS-14.0-arm64-arm-64bit
[codecarbon INFO @ 11:21:09]   Python version: 3.8.18
[codecarbon INFO @ 11:21:09]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 11:21:09]   Available RAM : 16.000 GB
[codecarbon INFO @ 11:21:09]   CPU count: 8
[codecarbon INFO @ 11:21:09]   CPU model: Apple M1
[codecarbon INFO @ 11:21:09]   GPU count: None
[codecarbon INFO @ 11:21:09]   GPU model: None


Epoch 1
-------------------------------
loss: 0.128699  [   64/60000]
loss: 0.197779  [ 6464/60000]
loss: 0.114261  [12864/60000]
loss: 0.252282  [19264/60000]
loss: 0.132137  [25664/60000]
loss: 0.245707  [32064/60000]
loss: 0.131120  [38464/60000]
loss: 0.282917  [44864/60000]
loss: 0.171711  [51264/60000]
loss: 0.291748  [57664/60000]
Test Error: 
 Accuracy: 94.3%, Avg loss: 0.198115 

Epoch 2
-------------------------------
loss: 0.127542  [   64/60000]
loss: 0.196985  [ 6464/60000]
loss: 0.113526  [12864/60000]
loss: 0.250835  [19264/60000]
loss: 0.131339  [25664/60000]
loss: 0.244800  [32064/60000]
loss: 0.130192  [38464/60000]
loss: 0.281431  [44864/60000]
loss: 0.170610  [51264/60000]
loss: 0.290222  [57664/60000]
Test Error: 
 Accuracy: 94.3%, Avg loss: 0.196992 

Epoch 3
-------------------------------
loss: 0.126397  [   64/60000]
loss: 0.196197  [ 6464/60000]
loss: 0.112792  [12864/60000]
loss: 0.249407  [19264/60000]
loss: 0.130554  [25664/60000]
loss: 0.243900  [32064/600

[codecarbon INFO @ 11:21:24] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:21:24] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 11:21:24] 0.000046 kWh of electricity used since the beginning.


loss: 0.248001  [19264/60000]
loss: 0.129777  [25664/60000]
loss: 0.243004  [32064/60000]
loss: 0.128384  [38464/60000]
loss: 0.278486  [44864/60000]
loss: 0.168480  [51264/60000]
loss: 0.287238  [57664/60000]
Test Error: 
 Accuracy: 94.5%, Avg loss: 0.194779 

Epoch 5
-------------------------------
loss: 0.124168  [   64/60000]
loss: 0.194678  [ 6464/60000]
loss: 0.111350  [12864/60000]
loss: 0.246605  [19264/60000]
loss: 0.129012  [25664/60000]
loss: 0.242067  [32064/60000]
loss: 0.127474  [38464/60000]
loss: 0.277020  [44864/60000]
loss: 0.167448  [51264/60000]
loss: 0.285780  [57664/60000]
Test Error: 
 Accuracy: 94.5%, Avg loss: 0.193687 

Epoch 6
-------------------------------
loss: 0.123091  [   64/60000]
loss: 0.193918  [ 6464/60000]
loss: 0.110650  [12864/60000]
loss: 0.245185  [19264/60000]
loss: 0.128227  [25664/60000]
loss: 0.241106  [32064/60000]
loss: 0.126572  [38464/60000]
loss: 0.275590  [44864/60000]
loss: 0.166459  [51264/60000]
loss: 0.284366  [57664/60000]
Test E

[codecarbon INFO @ 11:21:39] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:21:39] Energy consumed for all CPUs : 0.000042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 11:21:39] 0.000092 kWh of electricity used since the beginning.


...

Test Error: 
 Accuracy: 98.0%, Avg loss: 0.066397 

Epoch 497
-------------------------------
loss: 0.019307  [   64/60000]
loss: 0.036817  [ 6464/60000]
loss: 0.032697  [12864/60000]
loss: 0.017033  [19264/60000]
loss: 0.013880  [25664/60000]
loss: 0.026568  [32064/60000]
loss: 0.021997  [38464/60000]
loss: 0.045192  [44864/60000]
loss: 0.063017  [51264/60000]
loss: 0.041104  [57664/60000]
Test Error: 
 Accuracy: 98.0%, Avg loss: 0.066360 

Epoch 498
-------------------------------
loss: 0.019260  [   64/60000]
loss: 0.036674  [ 6464/60000]
loss: 0.032603  [12864/60000]
loss: 0.016963  [19264/60000]
loss: 0.013827  [25664/60000]
loss: 0.026487  [32064/60000]
loss: 0.021935  [38464/60000]
loss: 0.044985  [44864/60000]
loss: 0.062931  [51264/60000]
loss: 0.040866  [57664/60000]
Test Error: 
 Accuracy: 98.0%, Avg loss: 0.066331 

Epoch 499
-------------------------------
loss: 0.019237  [   64/60000]
loss: 0.036547  [ 6464/60000]
loss: 0.032546  [12864/60000]
loss: 0.016898  [19264/60000

[codecarbon INFO @ 11:58:55] Energy consumed for RAM : 0.003776 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:58:55] Energy consumed for all CPUs : 0.003147 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 11:58:55] 0.006922 kWh of electricity used since the beginning.


loss: 0.036434  [ 6464/60000]
loss: 0.032456  [12864/60000]
loss: 0.016846  [19264/60000]
loss: 0.013749  [25664/60000]
loss: 0.026318  [32064/60000]
loss: 0.021811  [38464/60000]
loss: 0.044571  [44864/60000]
loss: 0.062699  [51264/60000]
loss: 0.040440  [57664/60000]


[codecarbon INFO @ 11:58:59] Energy consumed for RAM : 0.003782 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:58:59] Energy consumed for all CPUs : 0.003152 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 11:58:59] 0.006934 kWh of electricity used since the beginning.


Test Error: 
 Accuracy: 98.0%, Avg loss: 0.066265 

Done!
0.002363178967208668
